In [34]:
from datetime import datetime
import datetime as dt
import time
from subgrounds.subgraph import SyntheticField, FieldPath
from subgrounds.subgrounds import Subgrounds
import pandas as pd
import duckdb as db
import IPython as ip
import  os as os

In [35]:
sg = Subgrounds()
apiKey = input('API Key, please')

In [36]:
#https://api.thegraph.com/subgraphs/name/corerouter/vesta-finance
vesta = sg.load_api('https://api.thegraph.com/subgraphs/name/corerouter/vesta-finance')

In [38]:
##this captures the list of people who borrowed VST against gOHM
vestaBorrowTicker = 0
vestaBorrowslist = pd.DataFrame()
vestaBorrowsDb = pd.DataFrame()
vestaBorrowListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (vestaBorrowTicker)*(1000)
    vestaBorrows = vesta.Query.borrows(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1' } #gOHM market

    )

    vestaBorrowslist = sg.query_df([
        vestaBorrows.id
        ,vestaBorrows.hash
        ,vestaBorrows.logIndex
        ,vestaBorrows.blockNumber
        ,vestaBorrows.timestamp
        ,vestaBorrows.account
        ,vestaBorrows.market
        ,vestaBorrows.market.rates
        ,vestaBorrows.asset
        ,vestaBorrows.amount
        ,vestaBorrows.amountUSD
        ,vestaBorrows.position
    ])

    #vestaBorrowslist.sort_values(by=['vestaBorrows_timestamp'])

    vestaBorrowsDb=pd.concat([vestaBorrowsDb, vestaBorrowslist])
    recordTimestamp1 = vestaBorrowsDb.iat[vestaBorrowTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    vestaBorrowsDbLength = len(vestaBorrowsDb)
    vestaBorrowListLength = len(vestaBorrowslist)
    recordID = vestaBorrowsDb.iat[vestaBorrowTicker,0]
    vestaBorrow = vestaBorrowsDb.iat[vestaBorrowTicker,1]
    print("iterations: ", vestaBorrowTicker, "Lines skipped: ",skipValue, "records collected: ", vestaBorrowListLength, " - latest recordID: ",vestaBorrow, " ", recordID , " - vestaBorrows DB length: ", vestaBorrowsDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", vestaBorrowTicker, "records collected: ", vestaBorrowListLength, " - userdb length: ", vestaBorrowsDbLength)
    vestaBorrowTicker = vestaBorrowTicker+1

    if daysAgo>daysLimit: exit =True
    if vestaBorrowListLength<1000: exit =True

vestaBorrowsDb.columns = vestaBorrowsDb.columns.str.replace("borrows", "transaction")
vestaBorrowsDb['event']='borrow'
vestaBorrowsDb['protocol']='vesta'
print("Done. Total Records collected: ", vestaBorrowsDbLength)

iterations:  0 Lines skipped:  0 records collected:  1000  - latest recordID:  0xebfdf5464a23bb3c12a019bbbaca75ceac4ea73a6cee465bf512825f85d3c8f1   borrow-0xebfdf5464a23bb3c12a019bbbaca75ceac4ea73a6cee465bf512825f85d3c8f1-45  - vestaBorrows DB length:  1000  - latest record from:  2023-02-13 08:57:03 0 days ago
iterations:  1 Lines skipped:  1000 records collected:  402  - latest recordID:  0x1b54e1c3ef70e868dbce6ee79aded1704c308e4d38f9a62f9802e4d221baf50b   borrow-0x1b54e1c3ef70e868dbce6ee79aded1704c308e4d38f9a62f9802e4d221baf50b-37  - vestaBorrows DB length:  1402  - latest record from:  2023-02-11 20:27:03 1 days ago
Done. Total Records collected:  1402


In [39]:
vestaBorrowsDb.head(100)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event,protocol
0,borrow-0xebfdf5464a23bb3c12a019bbbaca75ceac4ea...,0xebfdf5464a23bb3c12a019bbbaca75ceac4ea73a6cee...,45,60572308,1676296623,0xb9e291b68e584be657477289389b3a6deed3e34c,2,2,0,1,...,BORROWER,None,300000000000000000000,0,0,1,0,0,borrow,vesta
1,borrow-0x1b54e1c3ef70e868dbce6ee79aded1704c308...,0x1b54e1c3ef70e868dbce6ee79aded1704c308e4d38f9...,37,60125704,1676165223,0x6ae68eb2fa416fe091803fde014bbf7475468979,6,2,4,4,...,BORROWER,None,91776615499826319393959,0,0,1,0,0,borrow,vesta
2,borrow-0xa8530b80a3aae63661e62d05f9c83afc5e626...,0xa8530b80a3aae63661e62d05f9c83afc5e62634b5f4c...,45,60064234,1676142977,0xa13f3599ff620500f360dc6185ad30f625ebfc38,2,2,0,1,...,BORROWER,None,1614690000000000000000,0,0,1,0,0,borrow,vesta
3,borrow-0x0712c2c572f53f272b382289d178c5801205e...,0x0712c2c572f53f272b382289d178c5801205eb0b7c99...,45,59984711,1676119566,0x67fa1561c5f5631de08e38d4484e0df67872f626,2,2,0,1,...,BORROWER,None,6000370000000000000000,0,0,1,0,0,borrow,vesta
4,borrow-0xfa585079dbbd9dc2a051922131b8d6d5d65bb...,0xfa585079dbbd9dc2a051922131b8d6d5d65bbfa55a6a...,37,59820888,1676068023,0xc04f25ef37e2a960afa400e623c386625c7802e3,2,2,0,1,...,BORROWER,None,1350000084831034692182,0,0,2,0,0,borrow,vesta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,borrow-0x882593f140df3ff51511a47c7cb3c433e4a4c...,0x882593f140df3ff51511a47c7cb3c433e4a4c44a12c9...,34,48435322,1671865683,0xded5d8c6b271a48f3b9ba20e8aa761a6baa2fac9,5,1,4,4,...,BORROWER,None,160000052081071608108226,0,0,12,0,0,borrow,vesta
96,borrow-0xc502f9e5eb845a34d721e418dd597bb64e15a...,0xc502f9e5eb845a34d721e418dd597bb64e15abace157...,29,48288379,1671802945,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,152,0,152,132,...,BORROWER,None,0,0,0,1,2,0,borrow,vesta
97,borrow-0xd6835e5cc616cf65b8f46c3fb685c7d436206...,0xd6835e5cc616cf65b8f46c3fb685c7d4362061621bc4...,34,48094577,1671731058,0x804157889ac56a4e50278610c67960c1605a3264,16,2,14,24,...,BORROWER,None,11558591208330249968994,0,0,11,0,0,borrow,vesta
98,borrow-0xdbceaecbce8edc0d0d07d87f907a46c50c0e3...,0xdbceaecbce8edc0d0d07d87f907a46c50c0e33884ce3...,42,47726191,1671602208,0x73831e7c600f48cebbbff957b466be8e5c087f34,4,3,1,5,...,BORROWER,None,6423430000000000000000,0,0,1,0,0,borrow,vesta


In [40]:
##this tracks the ENTIRE list of events for vestaRepaying the VST repaid against gOHM
vestaRepayTicker = 0
vestaRepayslist = pd.DataFrame()
vestaRepaysDb = pd.DataFrame()
vestaRepayListLength = 1000
datediff = 0
now=0
daysAgo=0
daysLimit = 45
exit = False
while exit==False:
    skipValue = (vestaRepayTicker)*(1000)
    vestaRepays = vesta.Query.repays(
        orderBy='timestamp',
        orderDirection='desc',
        first=1000,
        skip = skipValue,
        where={ 'market': '0x8d9ba570d6cb60c7e3e0f31343efe75ab8e65fb1' } #gOHM market

    )

    vestaRepayslist = sg.query_df([
        vestaRepays.id
        ,vestaRepays.hash
        ,vestaRepays.logIndex
        ,vestaRepays.blockNumber
        ,vestaRepays.timestamp
        ,vestaRepays.account
        ,vestaRepays.market
        ,vestaRepays.market.rates
        ,vestaRepays.asset
        ,vestaRepays.amount
        ,vestaRepays.amountUSD
        ,vestaRepays.position
    ])

    #vestaRepayslist.sort_values(by=['vestaRepays_timestamp'])

    vestaRepaysDb=pd.concat([vestaRepaysDb, vestaRepayslist])
    recordTimestamp1 = vestaRepaysDb.iat[vestaRepayTicker,4]
    recordTimestamp = dt.datetime.fromtimestamp( recordTimestamp1 )
    now = (int(dt.datetime.utcnow().timestamp()))
    datediff=abs(int(now) - recordTimestamp1)
    daysAgo = int(datediff/86400)
    vestaRepaysDbLength = len(vestaRepaysDb)
    vestaRepayListLength = len(vestaRepayslist)
    recordID = vestaRepaysDb.iat[vestaRepayTicker,0]
    vestaRepay = vestaRepaysDb.iat[vestaRepayTicker,1]
    print("iterations: ", vestaRepayTicker, "Lines skipped: ",skipValue, "records collected: ", vestaRepayListLength, " - latest recordID: ",vestaRepay, " ", recordID , " - vestaRepays DB length: ", vestaRepaysDbLength, " - latest record from: ", recordTimestamp, daysAgo,"days ago")
    #print("iterations: ", vestaRepayTicker, "records collected: ", vestaRepayListLength, " - userdb length: ", vestaRepaysDbLength)
    vestaRepayTicker = vestaRepayTicker+1

    if daysAgo>daysLimit: exit =True
    if vestaRepayListLength<1000: exit =True

vestaRepaysDb.columns = vestaRepaysDb.columns.str.replace("repays", "transaction")
vestaRepaysDb['event']='Repay'
vestaRepaysDb['protocol']='vesta'
print("Done. Total Records collected: ", vestaRepaysDbLength)

iterations:  0 Lines skipped:  0 records collected:  1000  - latest recordID:  0xac8ecdaf59200e43e6e6233bcd31728b0db4bc5435932aef9921cf35ba9fc003   repay-0xac8ecdaf59200e43e6e6233bcd31728b0db4bc5435932aef9921cf35ba9fc003-26  - vestaRepays DB length:  1000  - latest record from:  2023-02-13 01:51:18 0 days ago
iterations:  1 Lines skipped:  1000 records collected:  993  - latest recordID:  0x9633a7166f3744cde872b630378d9eaeec5f0da82d7384673c164c12250486c0   repay-0x9633a7166f3744cde872b630378d9eaeec5f0da82d7384673c164c12250486c0-26  - vestaRepays DB length:  1993  - latest record from:  2023-02-13 01:44:00 0 days ago
Done. Total Records collected:  1993


In [41]:
vestaRepaysDb.head(10)

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event,protocol
0,repay-0xac8ecdaf59200e43e6e6233bcd31728b0db4bc...,0xac8ecdaf59200e43e6e6233bcd31728b0db4bc543593...,26,60476974,1676271078,0x6c01ec9622e50799d62ca076f2e2aa34dc840d5b,4,0,4,1,...,BORROWER,None,0,0,0,2,2,0,Repay,vesta
1,repay-0x9633a7166f3744cde872b630378d9eaeec5f0d...,0x9633a7166f3744cde872b630378d9eaeec5f0da82d73...,26,60475459,1676270640,0x5c9d09716404556646b0b4567cb4621c18581f94,5,0,5,1,...,BORROWER,None,0,0,0,2,3,0,Repay,vesta
2,repay-0x851007fa5a8371105a73452ccbd7a0a4eb976e...,0x851007fa5a8371105a73452ccbd7a0a4eb976e7d2c12...,32,59865436,1676083788,0xc9e166f2737e004e93800dcef0d608b014df07f6,25,4,21,16,...,BORROWER,None,3515000787053571080985,0,0,30,21,0,Repay,vesta
3,repay-0xa243d87a35b20328eac5637a6db98058707cd0...,0xa243d87a35b20328eac5637a6db98058707cd0161df6...,61,59508912,1675978810,0xf5d0318692f822680b733955828a25507f031c95,2,0,2,1,...,BORROWER,None,0,0,0,1,1,0,Repay,vesta
4,repay-0xdee0c294dd70fb50547428f7e6d6ea6b135cb8...,0xdee0c294dd70fb50547428f7e6d6ea6b135cb8a4d75a...,33,59130546,1675866794,0x515755b2c5a209976cf0de869c30f45ac7495a60,6,0,6,12,...,BORROWER,None,0,0,0,1,1,0,Repay,vesta
5,repay-0xd6e3c80447789b02b1b9282c1624d5885a19f4...,0xd6e3c80447789b02b1b9282c1624d5885a19f40712bd...,25,59010222,1675835228,0x7ee54ab0f204bb3a83df90fdd824d8b4abe93222,3,0,3,1,...,BORROWER,None,0,0,0,1,3,0,Repay,vesta
6,repay-0xae8658b03d6aee809d73702ece0d60ced1b6be...,0xae8658b03d6aee809d73702ece0d60ced1b6be240b82...,58,58773043,1675767985,0x09fc8cd1d422e77cef172ef85cda1953915a8b74,2,0,2,1,...,BORROWER,None,0,0,0,1,1,0,Repay,vesta
7,repay-0xa02ca5cb2651d3bd78de426c9c010eea013be5...,0xa02ca5cb2651d3bd78de426c9c010eea013be53c8843...,23,58627462,1675722780,0xfa4dfbe7b012b1d92fdd5544f30518bb58bd41ce,2,0,2,1,...,BORROWER,None,0,0,0,1,2,0,Repay,vesta
8,repay-0x44899a838a0d9eaf2f48a95594a3b1041bcbdf...,0x44899a838a0d9eaf2f48a95594a3b1041bcbdfabe482...,29,58627282,1675722712,0xfa4dfbe7b012b1d92fdd5544f30518bb58bd41ce,2,0,2,1,...,BORROWER,None,0,0,0,1,2,0,Repay,vesta
9,repay-0x91f5c48839b978c11fbbe3f9502c5538060257...,0x91f5c48839b978c11fbbe3f9502c55380602570911ec...,29,58336925,1675634888,0xe2a4de267cdd4ff5ed9ba13552f5c624b12db9b2,20,6,14,65,...,BORROWER,None,78903329999999999121255,0,0,6,1,0,Repay,vesta


In [47]:
balances = db.query(
    "Select "
    "   A.transaction_position_account_id "
    "   ,sum(A.transaction_amountUSD) totalUsdBorrowed "
    "   ,count(distinct A.transaction_id) borrow_transactions "
    "   ,sum(B.transaction_amountUSD) totalUsdRepaid "
    "   ,count(distinct B.transaction_id) repay_transactions "
    "   ,sum(A.transaction_amountUSD) -sum(B.transaction_amountUSD) netUsdBalance "
    "From vestaBorrowsDb A left outer join vestaRepaysDb B "
        "on A.transaction_position_account_id = B.transaction_position_account_id "
    "Group by 1 "
).df()
balances.head(100)

,transaction_position_account_id,totalUsdBorrowed,borrow_transactions,totalUsdRepaid,repay_transactions,netUsdBalance
0,0x6ae68eb2fa416fe091803fde014bbf7475468979,4.277411e+05,4,488255.761734,2,-6.051465e+04
1,0x15bae977db3b62ceb5a67921fae589fd80624e8d,1.048820e+08,4,596731.767138,150,1.042853e+08
2,0x7723c45fb37092ee65202db7a2fe562c5fe0ab48,1.125132e+04,5,6376.099291,6,4.875220e+03
3,0x515755b2c5a209976cf0de869c30f45ac7495a60,1.032425e+05,4,206364.919344,2,-1.031225e+05
4,0x394b67c6bc05abb14c73a57706dcd5cb85231c4e,2.581620e+03,3,2645.999633,1,-6.437964e+01
...,...,...,...,...,...,...
95,0xd54d23200e2bf7cb7a7d6792a55da1377a697799,1.246289e+05,6,56281.016513,9,6.834792e+04
96,0xa87870538f4b6589fb68f2e1612681fb505b450a,4.856250e+02,1,485.625003,1,0.000000e+00
97,0xc58ba8f7d5f0d08b93bfd66e59f87083ee6fae6b,1.042500e+03,1,1042.500006,1,0.000000e+00
98,0x777538e9e6fb692420ecd63add56ae2e34227e94,2.031004e+04,1,10155.021963,2,1.015502e+04


In [49]:
vestaTxnsDb = pd.concat([vestaBorrowsDb, vestaRepaysDb])
vestaTxnsDb.sort_values(by=['transaction_position_account_id', 'transaction_timestamp'])

,transaction_id,transaction_hash,transaction_logIndex,transaction_blockNumber,transaction_timestamp,transaction_account_id,transaction_account_positionCount,transaction_account_openPositionCount,transaction_account_closedPositionCount,transaction_account_depositCount,...,transaction_position_side,transaction_position_isCollateral,transaction_position_balance,transaction_position_depositCount,transaction_position_withdrawCount,transaction_position_borrowCount,transaction_position_repayCount,transaction_position_liquidationCount,event,protocol
50,borrow-0x91004cc1169082e83039e044446da5e0c99ac...,0x91004cc1169082e83039e044446da5e0c99ace028cdd...,42,54024990,1674103068,0x002525a77d262dd11ec63c9146a821b11564ce29,2,2,0,1,...,BORROWER,None,45000000000000000000000,0,0,1,0,0,borrow,vesta
380,borrow-0x82d29f62312491ce3d0f36dbdb69871897a48...,0x82d29f62312491ce3d0f36dbdb69871897a489e38240...,18,5699866,1644387457,0x002ee33092457900515a4249e3ec1cd2b2dc8c32,6,0,6,3,...,BORROWER,None,0,0,0,2,1,0,borrow,vesta
361,borrow-0xb57274061cd0023a1bfde66ef9752cfd409fd...,0xb57274061cd0023a1bfde66ef9752cfd409fdd3000e9...,9,5703973,1644391251,0x002ee33092457900515a4249e3ec1cd2b2dc8c32,6,0,6,3,...,BORROWER,None,0,0,0,2,1,0,borrow,vesta
985,repay-0xc6d2cc92892f2f959e7541e8ae2f2abee245bf...,0xc6d2cc92892f2f959e7541e8ae2f2abee245bf2ecfe3...,4,5744385,1644438194,0x002ee33092457900515a4249e3ec1cd2b2dc8c32,6,0,6,3,...,BORROWER,None,0,0,0,2,1,0,Repay,vesta
267,borrow-0x57f81166a353d23b2ee8e47b6748dfb90b628...,0x57f81166a353d23b2ee8e47b6748dfb90b62870f7aff...,18,5760221,1644455450,0x00ad3c523b715cbd6cc1771f1a1743269b220fa4,4,0,4,2,...,BORROWER,None,0,0,0,1,2,0,borrow,vesta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,repay-0x19c8abb34c314ca6cb0ca0d995f42316e20a39...,0x19c8abb34c314ca6cb0ca0d995f42316e20a39eb41b6...,4,28381570,1664911292,0xfd9e6fd73a0694057b632d8d3d4925d99c374306,2,2,0,2,...,BORROWER,None,41581000000000001716586,0,0,3,4,0,Repay,vesta
778,repay-0x30e20cd22584c4635ed3e9530c613cf5874f61...,0x30e20cd22584c4635ed3e9530c613cf5874f61c73185...,4,28382128,1664911724,0xfd9e6fd73a0694057b632d8d3d4925d99c374306,2,2,0,2,...,BORROWER,None,41581000000000001716586,0,0,3,4,0,Repay,vesta
936,borrow-0x5d5feffbd13fadea07888a07525e1f155c26a...,0x5d5feffbd13fadea07888a07525e1f155c26a03e9805...,18,10679246,1651079531,0xfda1215797d29414e588b2e62fc390ee2949aaaa,2,0,2,1,...,BORROWER,None,0,0,0,1,1,0,borrow,vesta
632,repay-0x04436ebdd2a880172ba72e85a43aa57b3b9a5f...,0x04436ebdd2a880172ba72e85a43aa57b3b9a5ff5875d...,3,10679749,1651079945,0xfda1215797d29414e588b2e62fc390ee2949aaaa,2,0,2,1,...,BORROWER,None,0,0,0,1,1,0,Repay,vesta


In [70]:
vestaTxnsSimpleDb = db.query(
    "with raw as (select "
        " transaction_account_id "
    ", transaction_position_id"
    ", transaction_timestamp "o'' \
                               ''
    ", event "
    ", transaction_amountUsd "
    ",Case when event = 'borrow' then transaction_amountUsd else -transaction_amountUsd end naturalAmount "
    "From vestaTxnsDb ) "
    "SELECT "
    "   * "
    ",sum(naturalAmount) over (partition by transaction_account_id order by transaction_timestamp asc) balance "
    "from raw ").df()
vestaTxnsSimpleDb.head(100)

,transaction_account_id,transaction_position_id,transaction_timestamp,event,transaction_amountUSD,naturalAmount,balance
0,0x002525a77d262dd11ec63c9146a821b11564ce29,0x002525a77d262dd11ec63c9146a821b11564ce29-0x8...,1674103068,borrow,45000.000000,45000.000000,45000.000000
1,0x002ee33092457900515a4249e3ec1cd2b2dc8c32,0x002ee33092457900515a4249e3ec1cd2b2dc8c32-0x8...,1644387457,borrow,10155.000000,10155.000000,10155.000000
2,0x002ee33092457900515a4249e3ec1cd2b2dc8c32,0x002ee33092457900515a4249e3ec1cd2b2dc8c32-0x8...,1644391251,borrow,5062.500000,5062.500000,15217.500000
3,0x002ee33092457900515a4249e3ec1cd2b2dc8c32,0x002ee33092457900515a4249e3ec1cd2b2dc8c32-0x8...,1644438194,Repay,15217.500000,-15217.500000,0.000000
4,0x0089c97abf56b6122de5d52d41eda521f188eb12,0x552860bef1197918b7496d15e731b1b9439cf456-0x8...,1664208961,Repay,0.010000,-0.010000,-0.010000
...,...,...,...,...,...,...,...
95,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0-0x8...,1667038797,borrow,20270.251996,20270.251996,466106.484725
96,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0-0x8...,1667039744,Repay,6494.008570,-6494.008570,459612.476156
97,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0-0x8...,1667130790,borrow,6076.748754,6076.748754,465689.224910
98,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0,0x0561a78021d8966ddd20c28c6c4318d8675ee1f0-0x8...,1667174058,borrow,10126.452891,10126.452891,475815.677801
